In [1]:
import pandas as pd
c1 = pd.read_csv("./sku_sales_c2.csv")
# c1.isnull().sum().sort_values(ascending=False) # 无空行

In [ ]:
# 绘图
import seaborn as sns
import matplotlib.pyplot as plt

plt.subplot(121)
sns.countplot(x='store_id', data=c1)
plt.subplot(122)
sns.countplot(x='sku_id', data=c1)
plt.show()

从这一步开始，对数据集进行预处理

In [2]:
import numpy as np
#Fetching the N most purchased products#
def N_most_labels(data, variable , N , all='TRUE'):
    vv, cc = np.unique(data[variable], return_counts=True)
    labels_freq_pd = np.column_stack((vv, cc))
    # labels_freq_pd = itemfreq(data[variable])
    labels_freq_pd = labels_freq_pd[labels_freq_pd[:, 1].argsort()[::-1]] #[::-1] ==> to sort in descending order
    if all == 'FALSE':
        main_labels = labels_freq_pd[:,0][0:N]
    else: 
        main_labels = labels_freq_pd[:,0][:]
    labels_raw_np = data[variable].values #transform in numpy
    labels_raw_np = labels_raw_np.reshape(labels_raw_np.shape[0],1)

    labels_filtered_index = np.where(labels_raw_np == main_labels)
    
    return labels_freq_pd, labels_filtered_index
# 修改N的值，可选出频次最高的N个商品(此处为100)

label_freq, labels_filtered_index111 = N_most_labels(data = c1, variable = "sku_id", N = 111, all='FALSE')
_, labels_filtered_index293 = N_most_labels(data = c1, variable = "sku_id", N = 293, all='FALSE')
_, labels_filtered_index1000 = N_most_labels(data = c1, variable = "sku_id", N = 1000, all='FALSE')
###
c1_filtered_111 = c1.loc[labels_filtered_index111[0],:] # 仅保存了销量在前N位的商品
c1_filtered_293 = c1.loc[labels_filtered_index293[0],:]
c1_filtered_1000 = c1.loc[labels_filtered_index1000[0],:]

merged_df2 = pd.merge(c1_filtered_293, c1_filtered_111, how='left', indicator=True)
merged_df3 = pd.merge(c1_filtered_1000, c1_filtered_293, how='left', indicator=True)
###
c1_filtered1 = c1_filtered_111
c1_filtered2 = merged_df2[merged_df2['_merge'] == 'left_only'].drop('_merge', axis=1)
c1_filtered3 = merged_df3[merged_df3['_merge'] == 'left_only'].drop('_merge', axis=1)
###
def get_month_year(df):
    df['month'] = df.date.apply(lambda x: x.split('-')[1])
    df['year'] = df.date.apply(lambda x: x.split('-')[0])

get_month_year(c1_filtered1)
###
c1_filtered1['date'] = pd.to_datetime(c1_filtered1['date'])
c1_filtered1['day'] = c1_filtered1['date'].dt.day_name()
###
c1_filtered1 = c1_filtered1.drop(['date', 'salable_status', 'year'], axis=1)
c1_filtered1.columns

Index(['store_id', 'sku_id', 'original_price', 'weather_type',
       'min_temperature', 'max_temperature', 'promotion_type', 'threshold',
       'discount_off', 'curr_day_ratio', 'quantity', 'month', 'day'],
      dtype='object')

In [ ]:
sum = 0
sp1 = 0
for i in range(len(label_freq)):
    sum += label_freq[i][1]
    if sum > 2064724/3*1:
        sp1=i
        break
print(sp1)

sp2 = 0
sum = 0
for i in range(len(label_freq)):
    sum += label_freq[i][1]
    if sum > 2064724*2/3:
        sp2=i
        break
print(sp2)

### one-hot encoding

In [3]:
dummy_variables = ['store_id', 'sku_id', 'month', 'day', 'promotion_type', 'weather_type']
# dummy_variables = ['month','day']
for var in dummy_variables:
    dummy = pd.get_dummies(c1_filtered1[var], prefix = var, drop_first = False)
    c1_filtered1 = pd.concat([c1_filtered1, dummy], axis = 1)

In [4]:
c1_filtered1 = c1_filtered1.drop(dummy_variables, axis = 1)

In [ ]:
col_c1_3 = c1_filtered1.columns.tolist()
col_c1_3 = [column for column in col_c1_3 if column != 'quantity']
file_path = 'col_c2_1.txt'
with open(file_path, 'w') as file:
    for column in col_c1_3:
        file.write(f"{column}\n")

### 划分训练、测试集(4/1)

In [ ]:
c1_filtered1 = c1_filtered1.reset_index(drop=True)  #we reset the index
y = c1_filtered1['quantity']
X = c1_filtered1.drop(['quantity'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=15)

## **Train**

### 线性回归

In [ ]:
from sklearn import linear_model

model = linear_model.LinearRegression()
results = model.fit(X_train, y_train)
print(results)

In [ ]:
print(results.intercept_, results.coef_)
print(results.coef_.shape)

In [ ]:
import statsmodels.api as sm
# import statsmodels.formula.api as sm
model = sm.OLS(np.array(y_train, dtype=float), np.array(X_train, dtype=float)) # 最小二乘线性回归
results = model.fit()
# Statsmodels gives R-like statistical output
results.summary()

# 线性回归效果很差 R=0.272

### mlp

In [ ]:
# Convert data as np.array
features = np.array(X_train, dtype=np.float32)
targets = np.array(y_train.values.reshape(y_train.shape[0],1), dtype=np.float32)
features_validation= np.array(X_test, dtype=np.float32)
targets_validation = np.array(y_test.values.reshape(y_test.shape[0],1), dtype=np.float32)

In [ ]:
print(features.shape)
print(targets.shape)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm 

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=256, 
                 hidden_dim3=128):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.norm1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.norm2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(hidden_dim2, hidden_dim3)
        self.norm3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(hidden_dim3, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.norm1(x)
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.norm2(x)
        x = torch.relu(self.fc3(x))
        x = self.norm3(x)
        x = self.fc4(x)
        return x

In [ ]:
# 划分训练集
X_train = torch.tensor(features, dtype=torch.float32).to("cuda:0")
y_train = torch.tensor(targets, dtype=torch.float32).to("cuda:0")
dataset = TensorDataset(X_train, y_train)
batch_size = 256  # 可根据需要调整批大小
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:

input_dim=19
model = MLP(input_dim)
model.to("cuda:0")
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-2)
num_epochs = 100


for epoch in range(num_epochs):
    total_loss = 0.0
    loop = tqdm(dataloader, leave = True, 
                initial=0, smoothing=0.01, 
                ncols=130, dynamic_ncols=False)
    for idx, (batch_X, batch_y) in enumerate(loop):
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")
    if (epoch % 10 == 0):
        torch.save(model.state_dict(), f"./checkpoint/mlp2_ep_{epoch}.pth")

# Train Loss降不下去

## LSTM
LSTM的训练集、测试集格式与其他模型不同，故此处进行重新整理

In [ ]:
from datetime import datetime

c1_filtered_lstm = c1.loc[labels_filtered_index[0],:] # 仅保存了销量在前N位的商品
# c1_filtered_lstm['date'] = pd.to_datetime(c1_filtered_lstm['date'])
c1_filtered_lstm['unix_date'] = c1_filtered_lstm['date'].apply(lambda x: datetime.timestamp(datetime.strptime(x, '%Y-%m-%d')))
c1_filtered_lstm = c1_filtered_lstm.drop(["date"], axis = 1)

In [ ]:
column_name = 'unix_date'

# 确保列名存在于DataFrame中
if column_name in c1_filtered_lstm:
    # 获取列的索引位置
    column_index = c1_filtered_lstm.columns.get_loc(column_name)
    # 重新排列列的顺序，将目标列移动到第一列
    c1_filtered_lstm = c1_filtered_lstm[ [column_name] + [col for col in c1_filtered_lstm.columns if col != column_name] ]

In [ ]:
# 独热编码
dummy_variables = ['store_id', 'sku_id']
# dummy_variables = ['month','day']
for var in dummy_variables:
    dummy = pd.get_dummies(c1_filtered_lstm[var], prefix = var, drop_first = False)
    c1_filtered_lstm = pd.concat([c1_filtered_lstm, dummy], axis = 1)
c1_filtered_lstm = c1_filtered_lstm.reset_index(drop=True)  #we reset the index

# 划分训练测试
y = c1_filtered_lstm['quantity']
X = c1_filtered_lstm.drop(['quantity'], axis = 1)
from sklearn.model_selection import train_test_split
num_test = 0.20
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X, y, test_size=num_test, random_state=15)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm 
features = np.array(X_train_lstm, dtype = np.float32)
targets = np.array(y_train_lstm.values.reshape(y_train_lstm.shape[0],1), dtype = np.float32)
features_validation= np.array(X_test_lstm, dtype = np.float32)
targets_validation = np.array(y_test_lstm.values.reshape(y_test_lstm.shape[0],1), dtype = np.float32)
X_train = torch.tensor(features).to("cuda:0")
y_train = torch.tensor(targets).to("cuda:0")

# 构造数据集及dataloader
dataset = TensorDataset(X_train, y_train)
batch_size = 256  # 可根据需要调整批大小
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item]

    def __len__(self):
        return len(self.data)


def nn_seq_us(dataset, B = 256):
    print('data processing...')
    # dataset = load_data()
    # split
    train = dataset[:int(len(dataset) * 0.6)]
    val = dataset[int(len(dataset) * 0.6):int(len(dataset) * 0.8)]
    test = dataset[int(len(dataset) * 0.8):len(dataset)]
    m, n = np.max(train[train.columns[1]]), np.min(train[train.columns[1]])

    def process(data, batch_size, shuffle):
        load = data[data.columns[1]]
        load = load.tolist()
        data = data.values.tolist()
        load = (load - n) / (m - n)
        seq = []
        for i in range(len(data) - 24):
            train_seq = []
            train_label = []
            for j in range(i, i + 24):
                x = [load[j]]
                train_seq.append(x)
            # for c in range(2, 8):
            #     train_seq.append(data[i + 24][c])
            train_label.append(load[i + 24])
            train_seq = torch.FloatTensor(train_seq)
            train_label = torch.FloatTensor(train_label).view(-1)
            seq.append((train_seq, train_label))

        # print(seq[-1])
        seq = MyDataset(seq)
        seq = DataLoader(dataset=seq, batch_size=batch_size, shuffle=shuffle, num_workers=0, drop_last=True)

        return seq

    Dtr = process(train, B, True)
    Val = process(val, B, True)
    Dte = process(test, B, False)

    return Dtr, Val, Dte, m, n


In [ ]:
Dtr, Val, Dte, m, n = nn_seq_us(c1_filtered_lstm, batch_size)

In [ ]:
import torch 
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.num_directions = 1 # 单向LSTM
        self.batch_size = batch_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input_seq):
        batch_size, seq_len = input_seq.shape[0], input_seq.shape[1]
        h_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to("cuda:0")
        c_0 = torch.randn(self.num_directions * self.num_layers, self.batch_size, self.hidden_size).to("cuda:0")
        output, _ = self.lstm(input_seq, (h_0, c_0))
        pred = self.linear(output)
        pred = pred[:, -1, :]
        return pred

In [ ]:
input_size = 124
hidden_size = 256
num_layers = 3
output_size = 1
model = LSTM(input_size=124, hidden_size=hidden_size, 
             num_layers=num_layers, output_size=output_size, 
             batch_size=batch_size)
model.to("cuda:0")

In [ ]:
from torch.optim.lr_scheduler import StepLR

def train(Dtr, Val, path):
    device = "cuda:0"
    input_size, hidden_size, num_layers = 1, 128, 2
    output_size = 1
    model = LSTM(input_size, hidden_size, num_layers, output_size, batch_size=256).to(device)

    loss_function = nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,
                                    weight_decay=1e-4)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    # training
    min_epochs = 10
    best_model = None
    min_val_loss = 5
    
    
    
    for epoch in range(100):
        train_loss = []
        loop = tqdm(Dtr, leave = True, 
                initial=0, smoothing=0.01, 
                ncols=130, dynamic_ncols=False)
        for idx, (seq, label) in enumerate(loop):
            seq = seq.to(device)
            label = label.to(device)
            y_pred = model(seq)
            loss = loss_function(y_pred, label)
            train_loss.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        scheduler.step()

        print('epoch {:03d} train_loss {:.8f}'.format(epoch, np.mean(train_loss)))
        model.train()
        
        if epoch%10==0:
            state = {'models': model.state_dict()}
            torch.save(state, path)

    


In [ ]:
train(Dtr, Val, path = './checkpoint/lstm_best.pth')

## XGBOOST

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(max_depth=25, tree_method="hist", device="cuda:0")

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)
train_pred = model.predict(X_train[0:600000])

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
print('R2 score using XG Boost= ',r2_score(y_test, y_pred), '/ 1.0')
print('MSE train loss using XG Boost= ',mean_squared_error(y_train[0:600000], train_pred), '/ 0.0')
print('MSE test loss using XG Boost= ',mean_squared_error(y_test, y_pred), '/ 0.0')

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(np.array(y_test[0:1000]), label = 'test')
plt.plot(np.array(y_pred[0:1000]), label = 'pred')
plt.legend()
plt.show()

In [ ]:
model.save_model("./xgboost_c1_1.pth")